In [1]:
import json
from collections import Counter
import pickle
import numpy as np
import pandas as pd
import os
import sys
import argparse
from os.path import dirname, realpath

import math
import hashlib
import datetime
import datetime as datetime
sys.path.append("./src/")

import torch
from torch import nn, optim
from torch.nn import functional as F

from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
sys.path.append(dirname(dirname(realpath(os.getcwd()))))

from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt


In [2]:
src_path = "G:\\FillmoreCancerData\\markhe\\VTERisk" 
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

def md5(key):
    return hashlib.md5(repr(key).encode()).hexdigest()
testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/fixed_dx.csv'))
pat_ids = (testDF['patient_id'] ).astype(int)
testDF['pids'] = pat_ids.apply(md5)

In [3]:
tdf = pd.DataFrame({'patient_id': (testDF['patient_id'] ),    
                    'pids': (testDF['pids'] ),    
                  'dob': (testDF['dob'] ),  
                  'outcome_date': (testDF['outcome_date'] ), 
                  'obs_time_end': (testDF['obs_time_end'] ),  
                  'index_date': (testDF['index_date'] ),  
                  'diag_date': (testDF['diag_date'] ),  
                  'outcome': (testDF['outcome'])    })

In [4]:
def logit (p): return np.log(  p / (1 - p) )
def expit (p): return   np.exp(p)/(1 + np.exp(p))

In [5]:
trainpath = os.path.join(src_path ,'logs_transformer_vte/L/1_15/', "28ae61cedbdb689ae7a33920009fefa2.results.train_preds")
testpath = os.path.join(src_path ,'logs_transformer_vte/L/1_15/', "28ae61cedbdb689ae7a33920009fefa2.results.test_preds")

with open(trainpath, 'rb') as f:
    R = pickle.load(f)
    p = np.array(R['probs'])

Df = pd.DataFrame.from_dict(R)
Df['probs'] = Df['probs'].astype(float)
Df['exams'] = Df['exams'].astype(int)
M = pd.merge(tdf, Df)
 

In [6]:
M.to_csv('CSV_for_R/totM_tr_1_15.csv') 


In [28]:
def get_4Ms(M):
    M1 = M.drop_duplicates(['patient_id'])
    
    m0 = M.duplicated(['patient_id']) 
    M234 = M[m0]
    M2 = M234.drop_duplicates(['patient_id'])

    m00 = M234.duplicated(['patient_id'])
    M34 = M234[m00]
    M3 = M34.drop_duplicates(['patient_id'])

    m000 = M34.duplicated(['patient_id'])
    M4 = M34[m000]
    M4 = M4.drop_duplicates(['patient_id'])
    return M1, M2, M3, M4

In [ ]:
M1, M2, M3, M4 = get_4Ms(M)

In [32]:
M1.to_csv('CSV_for_R/M1_tr_1_12.csv') 
M2.to_csv('CSV_for_R/M2_tr_1_12.csv') 
M3.to_csv('CSV_for_R/M3_tr_1_12.csv') 
M4.to_csv('CSV_for_R/M4_tr_1_12.csv') 

In [33]:
 with open(testpath, 'rb') as f:
    R_t = pickle.load(f)

Df_test = pd.DataFrame.from_dict(R_t)
Df_test['probs'] = Df_test['probs'].astype(float)
M_test = pd.merge(tdf, Df_test)
M_test1, M_test2, M_test3, M_test4 = get_4Ms(M_test)

In [34]:
M_test1.to_csv('CSV_for_R/M1_test_1_12.csv') 
M_test2.to_csv('CSV_for_R/M2_test_1_12.csv') 
M_test3.to_csv('CSV_for_R/M3_test_1_12.csv') 
M_test4.to_csv('CSV_for_R/M4_test_1_12.csv') 

##### Debug M4 why no duplicate

In [23]:
M1 = M.drop_duplicates(['patient_id'])

m0 = M.duplicated(['patient_id'])
M234 = M[m0]
M2 = M234.drop_duplicates(['patient_id'])

m00 = M234.duplicated(['patient_id'])
M34 = M234[m00]
M3 = M34.drop_duplicates(['patient_id'])

m000 = M34.duplicated(['patient_id'])
M4 = M34[m000]
M4 = M4.drop_duplicates(['patient_id'])


In [24]:
[ len(M), len(M1), len(M2), len(M3)]

[139274, 22371, 21045, 19548]

In [25]:
[ len(M234), len(M34), len(M4)]

[116903, 95858, 18400]

In [21]:
[   len(m0), len(m00), len(m000)]

[139274, 116903, 95858]

In [16]:
m000

2         False
3          True
4          True
5          True
6          True
          ...  
139265     True
139268    False
139269     True
139272    False
139273     True
Length: 95858, dtype: bool

In [12]:

m000 = M34.duplicated(['patient_id'])
M4 = M34[m000]


,patient_id,pids,dob,outcome_date,obs_time_end,index_date,diag_date,outcome,golds,probs,patient_golds,exams,dates,censor_times,days_to_final_censors
3,1014382133,f7fc19db953744e1a93461628a3dff72,1948-03-12,2016-11-09,2018-12-13,2016-10-21,2016-09-07,True,True,0.203854,True,170,2016-10-12T00:00:00,0,28
4,1014382133,f7fc19db953744e1a93461628a3dff72,1948-03-12,2016-11-09,2018-12-13,2016-10-21,2016-09-07,True,True,0.203854,True,170,2016-10-12T00:00:00,0,28
5,1014382133,f7fc19db953744e1a93461628a3dff72,1948-03-12,2016-11-09,2018-12-13,2016-10-21,2016-09-07,True,True,0.203854,True,170,2016-10-12T00:00:00,0,28
6,1014382133,f7fc19db953744e1a93461628a3dff72,1948-03-12,2016-11-09,2018-12-13,2016-10-21,2016-09-07,True,True,0.203854,True,170,2016-10-12T00:00:00,0,28
10,1017396330,c410728329b14dbc6d91a997a0caaebb,1945-11-10,2016-10-11,2016-10-11,2016-03-30,2016-01-14,False,False,0.224608,False,264,2016-03-30T00:00:00,0,195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139254,1007501555,4c94a9f0d6bcdc6f8bf42c095bdbaefa,1948-10-29,2018-02-03,2018-02-03,2015-04-09,2014-07-23,False,False,0.083903,False,401,2015-09-30T00:00:00,0,857
139255,1007501555,4c94a9f0d6bcdc6f8bf42c095bdbaefa,1948-10-29,2018-02-03,2018-02-03,2015-04-09,2014-07-23,False,False,0.036537,False,401,2015-12-22T00:00:00,0,774
139265,1002081914,459b9aa4ed385863ac6cd4799ac16a78,1957-02-22,2023-05-15,2023-05-15,2015-03-11,2015-02-05,False,False,0.034043,False,377,2015-11-02T00:00:00,0,2751
139269,1012230466,d31eddb331e0f204d1c3f3b8dd86abb2,1949-02-14,2023-05-25,2023-05-25,2013-09-16,2013-02-25,False,False,0.090705,False,401,2014-06-12T00:00:00,0,3269


In [ ]:

import sklearn
from sklearn.metrics import confusion_matrix
sklearn.metrics.roc_auc_score(tgold1,  tc1 , average = 'samples')

sklearn.metrics.roc_auc_score(tgold2,  tc2 , average = 'samples')

sklearn.metrics.roc_auc_score(tgold3,  tc3 , average = 'samples')

sklearn.metrics.roc_auc_score(tgold4,  tc4 , average = 'samples')


In [31]:
( confusion_matrix (tgold1,  tc1>.5) )

array([[6778, 8874],
       [  95,  385]], dtype=int64)

In [32]:
( confusion_matrix (tgold2,  tc2>.5) )

array([[11112,  2894],
       [  133,    87]], dtype=int64)

In [33]:
( confusion_matrix (tgold3,  tc3>.5) )

array([[12240,   436],
       [  139,    14]], dtype=int64)

In [34]:
( confusion_matrix (tgold4,  tc4>.5) )

array([[11578,     2],
       [  101,     0]], dtype=int64)

In [231]:
( confusion_matrix (tgold1,  tp1>.05) )

array([[ 1657, 13862],
       [   10,   496]], dtype=int64)

In [227]:
( confusion_matrix (tgold2,  tc2>.5) )

array([[12250,  1605],
       [  149,    50]], dtype=int64)

In [228]:
( confusion_matrix (tgold3,  tc3>.5) )

array([[12405,    42],
       [  130,     2]], dtype=int64)

In [229]:
( confusion_matrix (tgold4,  tc4>.5) )

array([[10584,   128],
       [   94,     4]], dtype=int64)

In [27]:
m1.to_csv('output/XCoh_Sen.csv') 

In [34]:
#m1.to_excel('output/dxMD_Sensitivity90.xlsx') 

In [39]:
y1 = np.array(M1['golds'],dtype = int)
x1 = M1['probs'].to_numpy()
platt.fit(x1, y1)
cal_1 = platt.predict(M_test1['probs'])